In [1]:
import os
import shutil
directory_out = './file/5.collate'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

directory_out

'./file/5.collate'

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [3]:
directory_in = './file/4.info'
list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = =  = = =  = = =

    print('Data Loading...')
    df_input = pd.read_excel(os.path.join(directory_in, file),
                             header=0,
                             dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)
    print('Loading Done !')
    print()

    # = = =  = = =  = = =

    list_series, list_column, list_column_note, list_column_note_2, list_column_price, list_column_specification = [], df_input.columns.tolist(), [], [], [], []
    for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
        dict_note, dict_note_2, dict_price, dict_dict = json.loads(series['Json_Note 1']), json.loads(series['Json_Note 2']), json.loads(series['Json_Price']), json.loads(series['Json_Specification'])
        dict_ = {k: v for dict_ in dict_dict.values() for k, v in dict_.items()}

        for key, value in dict_note.items():
            if key == '0':
                key = 'Note 1'
            else:
                key =  f'Note 1-{int(key) + 1}'

            if key not in list_column_note:
                list_column_note.append(key)

            series[key] = value

        # = = =  = = =  = = =

        for key, value in dict_note_2.items():
            if key == '0':
                key = 'Note 2'
            else:
                key =  f'Note 2-{int(key) + 1}'

            if key not in list_column_note_2:
                list_column_note_2.append(key)

            series[key] = value

        # = = =  = = =  = = =

        series['Price'] = dict_price.get('0', '')

        for key, value in dict_price.get('1', {}).items():
            key = f'Price-{int(key) + 2}'

            if key not in list_column_price:
                list_column_price.append(key)

            series[key] = value

        # = = =  = = =  = = =

        for key, value in dict_.items():
            if key in list_column:
                key = f'{key} (Specification)'

            if key not in list_column_specification:
                list_column_specification.append(key)

            series[key] = value

        list_series.append(series)

    # = = =  = = =  = = =

    df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                      ascending=[True],
                                                      ignore_index=True)
    df_output[list_column[:8] + sorted(list_column_note) + sorted(list_column_note_2) + [list_column[12], 'Price'] + sorted(list_column_price) + sorted(list_column_specification) + list_column[8:12] + list_column[13:]].to_excel(os.path.join(directory_out, file), index=False)

print()
print('Done ~')

1.xlsx
Data Loading...
Loading Done !



Progress: 100%|███████████████████████████| 353/353 [00:01<00:00, 222.92it/s]


2.xlsx
Data Loading...
Loading Done !



Progress: 100%|███████████████████████████| 401/401 [00:00<00:00, 450.97it/s]



Done ~


# 2. Vehicle

In [4]:
directory_in = './file/3.vehicle'
list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = =  = = =  = = =

    print('Data Loading...')
    df_input = pd.read_excel(os.path.join(directory_in, file),
                             header=0,
                             dtype=str).fillna('')

    df_input['Vehicle No'] = df_input['Vehicle No'].astype(int)
    df_input['Year'] = df_input['Year'].astype(int)
    print('Loading Done !')
    print()

    # = = =  = = =  = = =

    list_dict, grouped = [], df_input.groupby('JOIN_MPNTCPC')
    for mpntcpc, df in tqdm(grouped, desc='Progress', ncols=77):
        df = df.reset_index(drop=True)

        dict_vehicle = {}
        for i in range(len(df)):
            make, model, year = df.loc[i, 'Make'], df.loc[i, 'Model'], df.loc[i, 'Year']

            if make not in dict_vehicle:
                dict_vehicle[make] = {model: [year]}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            elif year not in dict_vehicle[make][model]:
                dict_vehicle[make][model].append(year)

        # = = =  = = =  = = =

        list_vehicle = []
        for make, dict_model in dict_vehicle.items():
            for model, list_year in dict_model.items():
                year_begin, year_end = min(list_year), max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = =  = = =  = = =

        list_dict.append({'JOIN_MPNTCPC': mpntcpc,
                          'Vehicle': '\n'.join(sorted(list_vehicle))})

    # = = =  = = =  = = =

    print('Data Outputting...')
    df_output = pd.DataFrame(list_dict).sort_values(by=['JOIN_MPNTCPC'],
                                                    ascending=[True],
                                                    ignore_index=True)
    df_output.to_excel(os.path.join(directory_out, file), index=False)
    print('Output Finished !')
    print()

# = = =  = = =  = = =

print('Done ~')

1-vehicle.xlsx
Data Loading...
Loading Done !



Progress: 100%|██████████████████████████| 353/353 [00:00<00:00, 1817.75it/s]

Data Outputting...


Output Finished !

2-vehicle.xlsx
Data Loading...
Loading Done !



Progress: 100%|██████████████████████████| 399/399 [00:00<00:00, 1913.06it/s]

Data Outputting...
Output Finished !

Done ~
